In [81]:
import pandas as pd
import numpy as np
import os

In [82]:
#1. Csrgar GRBs dsta base
#2.Genersr funcion de codigo
#3. generar .sh y batchsender


In [83]:
GRBsINFO='/lustre/hawcz01/scratch/userspace/jorgeamontes/grupo_grbs/kilonova_paper/maps/AutomatizedMaps/CSV_TXT/13final.csv'
PATH_GRBs_Healpix='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/healpix_maps/';label_1st='1';label_2nd='2'
PATH_GENERAL='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/'
GRBs=pd.read_csv(GRBsINFO)
global output
output='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes/output.txt'

In [ ]:
GRBsINFO='/lustre/hawcz01/scratch/userspace/jorgeamontes/grupo_grbs/kilonova_paper/maps/AutomatizedMaps/CSV_TXT/13final.csv'
PATH_GRBs_Healpix='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/healpix_maps/';label_1st='1';label_2nd='2'
PATH_GENERAL='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/'
GRBs=pd.read_csv(GRBsINFO)
global output
output='/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs/codes/output.txt'
def CODE(grb,Ra,Dec,bin_Size,path,label):
    PATH_GRBs=path+'/'+grb+'/'
    os.makedirs(PATH_GENERAL+'/HISTOGRAMs_ALL_GRBs',exist_ok=True)
    hist_path=PATH_GENERAL+'/HISTOGRAMs_ALL_GRBs'
    print(hist_path)
    bin_size=max(bin_Size,GRBs[GRBs['Name']==grb]['Error_Radius'].values[0])
    code=f"""#! /usr/bin/env bash
#SBATCH --mem-per-cpu=4GB
#SBATCH -o {PATH_GRBs}/{grb}/maps/codes/HIST.out
#SBATCH --mail-type=all
#SBATCH --mail-user=smfraijac@astro.unam.mx
#SBATCH -J UL{grb}
#Author: Sara_Fraija
source /data/disk01/home/smfraijac/.bashrc
hawc_aerie
Label=1
GRB={grb}_${{Label}}
RA={Ra}
DEC={Dec}
Input_path={PATH_GRBs}
Hist_path={hist_path}
Bin_size={bin_size}
output_file={hist_path}/{grb}_PSF_{bin_size}_Hist${{Label}}.txt
max_output_file={PATH_GENERAL}/MaxSigHist_{bin_size}_${{Label}}.txt
rm -f output_file
# makeSignificanceHistogram.py {PATH_GRBs}/transit_1/{grb}_transit_1.fits.gz --ra {Ra} --dec {Dec} --includeArea --binsize {bin_size} --batch -o {hist_path}/{grb}.png>> $output_file
PL=$(makeSignificanceHistogram.py {PATH_GRBs}/transit_1/{grb}_transit_1.fits.gz --ra ${{RA}} --dec ${{DEC}} --includeArea --batch --binsize ${{Bin_size}} -o ${{Hist_path}}/${{GRB}}.png)
echo -e ${{PL}}" > $output_file\n
Maxi=$(grep "Max" ${{output_file}})
echo -e "{grb},${{Label}},${{Maxi}}" >> $max_output_file
####################################################
Label_2=2
GRB={grb}_${{Label_2}}
Hist_path={hist_path}
Bin_size={bin_size}
output_file={hist_path}/{grb}_PSF_{bin_size}_Hist${{Label_2}}.txt
max_output_file={hist_path}/MaxSigHist_{bin_size}_${{Label_2}}.txt
rm -f output_file
# makeSignificanceHistogram.py {PATH_GRBs}/transit_2/{grb}_transit_2.fits.gz --ra {Ra} --dec {Dec} --includeArea --binsize {bin_size} --batch -o {hist_path}/{grb}.png>> $output_file
PL=$(makeSignificanceHistogram.py {PATH_GRBs}/transit_2/{grb}_transit_2.fits.gz --ra ${{RA}} --dec ${{DEC}} --includeArea --batch --binsize ${{Bin_size}} -o ${{Hist_path}}/${{GRB}}.png)
echo -e ${{PL}}" > $output_file\n
Maxi=$(grep "Max" ${{output_file}})
echo -e "{grb},${{Label_2}},${{Maxi}}" >> $max_output_file
"""
    return(code)
def file_creator(CONTENT,GRB,file_path,label,bin_size):
    file=PATH_GENERAL+f'/codes/HistogramSig{GRB}'+label+str(bin_size)+'.sh'
    with open(file,'w') as file2:
        file2.write(CONTENT)
def Nose(bin_Size,path,label):
    for grb in GRBs['Name']:
        Ra=GRBs[GRBs['Name']==grb]['Ra'].values[0]
        Dec=GRBs[GRBs['Name']==grb]['Dec'].values[0]
        code=CODE(grb,Ra,Dec,bin_Size,path,label)
        file_creator(code,grb,path,label,bin_Size) 
def BatchSender(PATH_grbs,label,bin_size):
    sbatch='sh '
    Path2=f'/codes/HistogramSig{GRB}'+label+str(bin_size)+'.sh'
    file_path=PATH_GENERAL+f'/BatchSenderHistogram'+str(bin_size)+'.sh'
    List=[sbatch+PATH_grbs+'/'+i+Path2 +'\n'  for i in GRBs['Name']]
    os.remove(file_path)
    with open(file_path, 'w+') as bash_file:     
        bash_file.write('rm -f {PATH_GENERAL}/MaxSigHist_{bin_size}_{label}.txt')  
        for i in List:
          bash_file.write(i)                 
# def remove(grb,path):
#     PATH_GRBs=path+'/'+grb+'/maps/Healpix'
#     os.rmdir(PATH_GRBs+'/HISTOGRAMs_ALL_GRBs')  
Nose(0.6,PATH_GRBs_Healpix,label_1st)
Nose(0.6,PATH_GRBs_Healpix,label_2nd)
# BatchSender(PATH_GENERAL,label_2nd,0.6)
# Nose(0.2,PATH_GRBs_2nd_Transit,label_2nd)                                                                                                             
# Nose(0.1,PATH_GRBs_1st_Transit,label_1st)
# Nose(0.1,PATH_GRBs_2nd_Transit,label_2nd)
# Nose(0.05,PATH_GRBs_1st_Transit,label_1st)
# Nose(0.05,PATH_GRBs_2nd_Transit,label_2nd)
# for i in [0.05,0.1,0.2]:
#     BatchSender(PATH_GRBs_1st_Transit,label_1st,i)
    # BatchSender(PATH_GRBs_2nd_Transit,label_2nd,i)


                                                                                                                                             


/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_ALL_GRBs
/lustre/hawcz01/scratch/userspace/jorgeamontes/GRB_KN/data/ULs//HISTOGRAMs_A

In [85]:
grb='GRB150110923'
GRBs[GRBs['Name']==grb]['Error_Radius'].values[0]

0.05